In [ ]:
%run preprocecssing_helpers.ipynb
import os

import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
path = 'F:\StroopEEG'
eeglab_files = os.listdir(path)

In [ ]:
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files

In [ ]:
eeglab_files[0]

In [ ]:
# read file
raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)

In [ ]:
picks = mne.pick_types(raw.info, meg = False, eeg= True, eog= False, stim = False)


In [ ]:
raw.plot();

In [ ]:
raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
raw.filter(5,100, fir_design='firwin')

In [ ]:
ica = ICA(n_components=None, max_iter = 1000, random_state = 42)

In [ ]:
ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))

In [ ]:
ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                     ecg_score_func='pearsonr', skew_criterion=None, 
                     kurt_criterion=None, var_criterion=None,add_nodes=None)

In [ ]:
ica.plot_sources(raw);

In [ ]:
ica.plot_overlay(raw, start = 0, stop = 100);

In [ ]:
raw_ica = ica.apply(raw)

In [ ]:
raw_ica.plot(duration=20,  n_channels=128);

In [ ]:
m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)

In [ ]:
raw.add_events = m_event

In [ ]:
# event_id : this dictionary is created from information from Dave's dissertation page 27
concepts={1:'BABY',2:'BELL',3:'BIRD',4:'BURP',5:'DOG',6:'DRUM',
              7:'KNOCK',8:'LAUGH',9:'PHONE',10:'TRAIN',11:'WATER'}

event_id = {'trail_start':  31, 
            'left_button':  131, 
            'right_button': 132,
            'congruent':    133,
            'incongruent':  134,
            'correct':      129,
            'incorrect':    130,
            # targets
            't_baby' :        135,
            't_bell':         136,
            't_bird':         137,
            't_burp':         138,
            't_dog':          139,
            't_drum':         140,
            't_knock':        141,
            't_laugh':        142,
            't_phone':        143,
            't_train':        144,
            't_water':        145,
            #flankers
            'f_baby' :        155,
            'f_bell':         156,
            'f_bird':         157,
            'f_burp':         158,
            'f_dog':          159,
            'f_drum':         160,
            'f_knock':        161,
            'f_laugh':        162,
            'f_phone':        163,
            'f_train':        164,
            'f_water':        165,
}
# event codes 1 - 24 represent flanker and target stim codes. They are NOT explicity defined in dissertation.
for i in range(1,13):
    event_id["flanker_stim_{0}".format(i)] = i
    
for j in range(13, 25):
    event_id["target_stim_{0}".format(j)] = j

In [ ]:
epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True, )

In [ ]:
epoch.resample(256);

In [ ]:
epoch.plot(n_channels=20, n_epochs=20, title="epochs")